# 第8章 IO库

之前我们已经使用过很多IO库设施：
- istream（输入流）类型，提供输入操作
- ostream（输出流）类型，提供输出操作
- cin，一个istream对象，从标准输入读取数据
- cout，一个ostream对象，向标准输出写入数据
- cerr，一个ostream对象，通常用于输出程序错误消息，写入到标准错误
- `>>`运算符，用来从一个istream对象读取输入数据
- `<<`运算符，用来向一个ostream对象写入输出数据
- getline函数，从一个给定的istream读取一行数据，存入一个给定的string对象中

## 8.1 IO类

- 标准库定义的IO类型分布在三个头文件中：iostream定义了用于读写流的基本类型，fstream定义了读写命名文件的类型，sstream定义了读写内存string对象的类型
- 设备类型和字符大小都不会影响我们要执行的IO操作，例如，我们可以用`>>`读取数据，而不用管是从一个控制台窗口，一个磁盘文件，还是一个string读取；类似地，我们也不用管读取的字符能存入一个char对象内，还是需要一个wchar_t对象来存储
- 由于fstream和sstream均继承自iostream，本节剩下部分所介绍的标准库流特性都可以无差别地应用于普通流、文件流和string流，以及char或宽字符流版本

### 8.1.1 IO对象无拷贝或赋值

- 我们不能拷贝或对IO对象赋值
```c++
ofstream out1, out2;
out1 = out2; // 错误：不能对流对象赋值
ofstream print(ofstream); // 错误：不能初始化ofstream参数
out2 = print(out1); // 不能拷贝对象
```
- 由于不能拷贝IO对象，因此我们也不能将形参或者返回类型设置为流类型，进行IO操作的函数通常以引用方式传递和返回流，读写一个IO对象会改变其状态，因此传递和返回的引用不能是const的

### 8.1.2 条件状态

- 一个流一旦发生错误，其上后续的IO操作都会失败，只有当一个流处于无错状态时，我们才可以从它读取数据，向它写入数据；由于流可能处于错误状态，因此代码通常应该在使用一个流之前检查它是否处于良好状态，确定一个流对象的状态的最简单的方法是将它当作一个条件来使用
```c++
// 如果输入操作成功，流保持有效状态，则条件为真
while (cin >> word) {
    // ok: 读取成功...
}
```

### 8.1.3 管理输出缓冲

- 每个输出流都管理一个缓冲区，用来保存程序读写的数据；有了缓冲机制，操作系统就可以将程序的多个输出操作合成单一的系统级写操作，这可以带来性能提升
- 导致缓冲刷新（即，数据真正写到输出设备或文件）的原因有很多：1.程序正常结束，作为main函数的return操作的一部分，缓冲刷新被执行；2.缓冲区满时，需要刷新缓冲，而后新的数据才能继续写入缓冲区；3.使用操纵符如endl来显式刷新缓冲区；4.在每个输出操作之后，可以用操纵符unitbuf设置流的内部状态，来清空缓冲区，默认情况下，对cerr是设置unitbuf的，因此写到cerr的内容都是立即刷新的；5.一个输出流可能被关联到另一个流，这样当读写被关联的流时，关联到的流的缓冲区会被刷新，例如，默认情况下，cin和cerr都关联到cout，因此读cin或写cerr都会导致cout的缓冲区被刷新

- 如果程序异常终止，输出缓冲区是不会被刷新的，这就意味着，当一个程序崩溃后，它所输出的数据很可能停留在输出缓冲区中等待打印

#### 刷新输出缓冲区

- 我们已经使用过操纵符endl，它完成换行并刷新缓冲区的工作；IO库中还有两个类似的操纵符：flush和ends；flush刷新缓冲区，但不输出任何额外的字符，ends向缓冲区插入一个空字符，然后刷新缓冲区

#### unitbuf

- 如果想在每次输出操作后都刷新缓冲区，我们可以使用unitbuf操纵符，它告诉流在接下来的每次写操作后都进行一次flush操作，而unitbuf操纵符则重置流，使其恢复使用正常的系统管理的缓冲区刷新机制
```c++
cout << unitbuf; // 所有输出操作后都会立即刷新缓冲区
cout << nounitbuf; // 回到正常的缓冲方式
```

#### 关联输入和输出流

- 当一个输入流被关联到一个输出流时，任何试图从输入流读取数据的操作都会先刷新关联的输出流，标准库将cout和cin关联到一起
- 交互式系统通常应该关联输入流和输出流，这意味着所有输出，包括用户提示信息，都会在读操作之前被打印出来

## 8.2 文件输入输出

- 头文件fstream定义了三个类型来支持文件IO：ifstream从一个给定文件读取数据，ofstream向一个给定文件写入数据，以及fstream可以读写给定文件
- 这些类型提供的操作与我们之前已经使用过的对象cin和cout的操作一样，特别是，可以用IO运算符（<< 和 >>）来读写文件，可以用getline从一个ifstream读取数据

### 8.2.1 使用文件流对象

- 当我们想要读写一个文件时，可以定义一个文件流对象，并将对象与文件关联起来，每个文件流类都定义了一个名为open的成员函数，它完成一些系统相关的操作，来定位给定的文件，并视情况打开为读或写模式
```c++
ifstream in(infile); // open自动被调用
ofstream out; // 输出文件流未关联到任何文件
```

- 当一个fstream对象离开其作用域时，会被销毁，close会自动被调用，与之关联的文件会自动关闭

### 8.2.2 文件模式

- 每个流都有一个关联的文件模式（file mode），用来指出如何使用文件
- 无论用哪种方式打开文件，我们都可以指定文件模式，调用open打开文件时可以，用一个文件名初始化流来隐式打开文件时也可以
- 每个文件流类型都定义了一个默认的文件模式，当我们未指定文件模式时，就使用此默认模式：与ifstream关联的文件默认以in模式打开，与ofstream关联的文件默认以out模式打开，与fstream关联的文件默认以in和out模式打开
- 默认情况下，当我们打开一个ofstream时，文件的内容会被丢弃，阻止一个ofstream清空给定文件内容的方法是同时指定app模式

## 8.3 string流

- sstream头文件定义了三个类型来支持内存IO，这些类型可以向string写入数据，从string读取数据，就像string是一个IO流一样
- 当我们的某些工作是对整行文本进行处理，而其他一些工作是处理行内的单个单词时，通常可以使用istringstream
- 当我们逐步构造输出，希望最后一起打印时，ostringstream是很有用的

## 小结

C++使用标准库类来处理面向流的输入和输出：
- iostream处理控制台IO
- fstream处理命名文件IO
- stringstream完成内存string的IO